In [31]:
import pymysql
import requests
import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy import create_engine
import time
import logging
import boto3
import pendulum
import pytz
import os, shutil
from io import StringIO
from tqdm import tqdm
import swifter

from keys import *


In [32]:
# today = datetime.now().strftime("%Y%m%d")
today = "20240902"
types = {'Name':'str','Code':'str'}
code_df = pd.read_csv("./code.csv", dtype=types).rename(columns={"Name":'name',"Code":"stock_code"})

In [33]:
df_types = {'stck_bsop_date' : "str",
'stck_clpr' : "int",
'stck_oprc' : "int",
'stck_hgpr' : "int",
'stck_lwpr' : "int",
'acml_vol' : "int",
'acml_tr_pbmn' : "int",
'flng_cls_code' : "int",
'prtt_rate' : "float",
'mod_yn' : "str",
'prdy_vrss_sign' : "int",
'prdy_vrss' : "int",
'revl_issu_reas' : "str",
'hts_avls' : "int",
'prdy_vol' : "int",
'stock_code' : "str"}
# s3랑 연결 설정
access = aws_access_key() # git 올릴 때를 위한 암호화 
secret = aws_secret()

s3 = boto3.client(
    's3',
    aws_access_key_id= access, 
    aws_secret_access_key= secret,
    region_name='ap-northeast-2'
)

bucket_name = 'antsdatalake'
folder = 'once_time/' 

# rds와 연결
user = 'ants'
password = rds_password()
host= end_point()
port = 3306
database = 'datawarehouse'
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

conn = pymysql.connect(host=host, user=user, passwd=password, db=database)
cursor = conn.cursor()

In [34]:
# df = pd.read_csv(f"~/airflow/stock_data/data/{today}.csv", dtype=df_types)
# df.drop(columns=['stck_oprc','acml_vol', 'stck_hgpr','stck_lwpr','acml_vol','acml_tr_pbmn','flng_cls_code','prtt_rate','mod_yn','prdy_vrss_sign','prdy_vrss','revl_issu_reas'], inplace=True)
# df.rename(columns={"stck_bsop_date":"date","stck_clpr":"closing_price","hts_avls":"hts_total","prdy_vol":"prev_trading"}, inplace=True)
# df = pd.merge(df, code_df, on = 'stock_code', how='left')

# df_columns = ['stock_code', 'name', 'date', 'closing_price', 'hts_total', 'prev_trading']
# upload_df = df[df_columns]
# upload_df['date'] = pd.to_datetime(upload_df['date'], format="%Y%m%d")

# upload_df.to_sql('once_time', index=False, if_exists="append", con=engine)

In [35]:
# select_sql = "SELECT * FROM `once_time` WHERE `date` >= CURDATE() - INTERVAL 180 DAY;"
# rds_df = pd.read_sql(select_sql, conn)
# rds_df['date'] = pd.to_datetime(rds_df['date'])
# rds_df = rds_df.sort_values(by=['stock_code', 'date'])
# print("read 완료")

# # 업데이트할 날짜
# target_date = pd.Timestamp(f'{today}')

# if rds_df.empty:
#     print(f"No data available up to {target_date}")
# else:
#     if 'closing_price' in rds_df.columns:
#         # 이동 평균 계산
#         rds_df['MA5'] = round(rds_df.groupby('stock_code')['closing_price'].transform(lambda x: x.rolling(window=5, min_periods=1).mean()),1)
#         rds_df['MA20'] = round(rds_df.groupby('stock_code')['closing_price'].transform(lambda x: x.rolling(window=20, min_periods=1).mean()),1)
#         rds_df['MA60'] = round(rds_df.groupby('stock_code')['closing_price'].transform(lambda x: x.rolling(window=60, min_periods=1).mean()),1)
#         rds_df['MA120'] = round(rds_df.groupby('stock_code')['closing_price'].transform(lambda x: x.rolling(window=120, min_periods=1).mean()),1)

#         # 특정 날짜의 데이터만 업데이트
#         for index, row in rds_df[rds_df['date'] == target_date].iterrows():
#             update_sql = """
#                 UPDATE `once_time` 
#                 SET MA5 = %s, MA20 = %s, MA60 = %s, MA120 = %s
#                 WHERE stock_code = %s AND `date` = %s
#             """
#             cursor.execute(update_sql, (
#                 row['MA5'], row['MA20'], row['MA60'], row['MA120'],
#                 row['stock_code'], row['date']
#             ))
#             print(row)
#         conn.commit()

# print("update 완료")

# # 데이터베이스 연결 종료
# cursor.close()
# conn.close()

In [36]:
select_sql = "SELECT * FROM `once_time` WHERE `date` >= CURDATE() - INTERVAL 180 DAY;"
rds_df = pd.read_sql(select_sql, conn)
rds_df['date'] = pd.to_datetime(rds_df['date'])
rds_df = rds_df.sort_values(by=['stock_code', 'date'])
print("read 완료")

# 업데이트할 날짜
target_date = pd.Timestamp(f'{today}')
rds_df[rds_df['date'] == target_date]

/tmp/ipykernel_547374/1761553906.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rds_df = pd.read_sql(select_sql, conn)


read 완료


,stock_code,name,date,closing_price,hts_total,prev_trading,MA5,MA20,MA60,MA120
225303,000020,동화약품,2024-09-02,8000,2235,41121,8000.0,8000.0,8000.0,8000.0
224267,000040,KR모터스,2024-09-02,586,352,232291,586.0,586.0,586.0,586.0
224259,000050,경방,2024-09-02,6540,1793,1201246,6540.0,6540.0,6540.0,6540.0
225954,000070,삼양홀딩스,2024-09-02,72300,6192,5446,72300.0,72300.0,72300.0,72300.0
225236,000080,하이트진로,2024-09-02,20650,14483,165860,20650.0,20650.0,20650.0,20650.0
...,...,...,...,...,...,...,...,...,...,...
224866,950170,JTC,2024-09-02,4590,2375,51576,4590.0,4590.0,4590.0,4590.0
226205,950190,고스트스튜디오,2024-09-02,10700,1453,30380,10700.0,10700.0,10700.0,10700.0
225499,950200,소마젠,2024-09-02,4415,849,113296,4415.0,4415.0,4415.0,4415.0
225918,950210,프레스티지바이오파마,2024-09-02,13750,8263,60652,13750.0,13750.0,13750.0,13750.0


In [37]:
# 벡터화된 방식으로 DataFrame에 MA 컬럼 추가
rds_df['MA5'] = rds_df.groupby('stock_code')['closing_price'].transform(lambda x: x.rolling(window=5, min_periods=1).mean()).round(1)
rds_df['MA20'] = rds_df.groupby('stock_code')['closing_price'].transform(lambda x: x.rolling(window=20, min_periods=1).mean()).round(1)
rds_df['MA60'] = rds_df.groupby('stock_code')['closing_price'].transform(lambda x: x.rolling(window=60, min_periods=1).mean()).round(1)
rds_df['MA120'] = rds_df.groupby('stock_code')['closing_price'].transform(lambda x: x.rolling(window=120, min_periods=1).mean()).round(1)

# 특정 날짜의 데이터만 추출

print("계산 완료")

계산 완료


In [38]:
target_date

Timestamp('2024-09-02 00:00:00')

In [39]:
subset_df = rds_df[rds_df['date'] == target_date]
update_sql = """
    UPDATE `once_time` 
    SET MA5 = %s, MA20 = %s, MA60 = %s, MA120 = %s
    WHERE stock_code = %s AND `date` = %s
"""

# 데이터베이스 커넥션 및 커서 오픈
with conn.cursor() as cursor:
    data = [
        (row['MA5'], row['MA20'], row['MA60'], row['MA120'], row['stock_code'], row['date']) 
        for _, row in subset_df.iterrows()
    ]
    cursor.executemany(update_sql, data)
    conn.commit()
    target_date += timedelta(days=1)
    print(target_date)

2024-09-03 00:00:00


In [40]:
# for x in range(5):
#     subset_df = rds_df[rds_df['date'] == target_date]
#     update_sql = """
#         UPDATE `once_time` 
#         SET MA5 = %s, MA20 = %s, MA60 = %s, MA120 = %s
#         WHERE stock_code = %s AND `date` = %s
#     """

#     # 데이터베이스 커넥션 및 커서 오픈
#     with conn.cursor() as cursor:
#         data = [
#             (row['MA5'], row['MA20'], row['MA60'], row['MA120'], row['stock_code'], row['date']) 
#             for _, row in subset_df.iterrows()
#         ]
#         cursor.executemany(update_sql, data)
#         conn.commit()
#         target_date += timedelta(days=1)
#         print(target_date)

In [41]:
# index_sql = """
#     CREATE INDEX idx_stock_date ON once_time(stock_code(6), `date`);
# """
# cursor.execute(index_sql)